In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.deterministic import DeterministicProcess
from xgboost import XGBRegressor



store_sales = pd.read_csv(
    '/kaggle/input/store-sales-time-series-forecasting/train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales','onpromotion'],
    parse_dates=['date']
)
store_sales['date'] = store_sales.date.dt.to_period('D')
store_sales = store_sales.set_index(['store_nbr', 'family', 'date']).sort_index()

def tsf(df):
    df=df.copy()
    df['day'] = df.index.day
    df['month'] = df.index.month
    df['dow'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['doy'] = df.index.dayofyear
    return(df)

y = store_sales.unstack(['store_nbr', 'family']).loc["2014-01-01":"2017-08-15"]
y1 = y.drop('onpromotion',axis=1)

dp = DeterministicProcess(
     index=y1.index,  
     constant=True,  
     order=2,        
     drop=True
)
X1 = dp.in_sample()

model = LinearRegression(fit_intercept=False)
model.fit(X1, y1)
y_fit = pd.DataFrame(model.predict(X1), index=X1.index, columns=y1.columns)


#STORE_NBR = 3  # 1 - 54
#FAMILY = 'PRODUCE'
# Uncomment to see a list of product families
# display(store_sales.index.get_level_values('family').unique())

#ax = y1['sales', STORE_NBR, FAMILY].plot()
#ax = y_fit['sales', STORE_NBR, FAMILY].plot()
#ax.set_title(f'{FAMILY} Sales at Store {STORE_NBR}');

y_resid = y1 - y_fit.squeeze()

X2 = y.drop('sales',axis=1)

X2 = tsf(X2)

#!nvidia-smi
#View GPU characteristics

xgb = XGBRegressor(random_state=35643419,tree_method='hist',device='cuda')
#To fix mismatch between data location
xgb.set_params(predictor='gpu_predictor')
xgb.fit(X2, y_resid)

y_fit_boosted = xgb.predict(X2) + y_fit.squeeze()

#STORE_NBR = 2  # 1 - 54
#FAMILY = 'PRODUCE'
# Uncomment to see a list of product families
# display(store_sales.index.get_level_values('family').unique())

#ax = y1['sales', STORE_NBR, FAMILY].plot(color="blue")
#ax = y_fit_boosted['sales', STORE_NBR, FAMILY].plot(color="orange")
#ax.set_title(f'{FAMILY} Sales at Store {STORE_NBR}');

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_ex

In [2]:
df_test = pd.read_csv(
    '/kaggle/input/store-sales-time-series-forecasting/test.csv',
    parse_dates=['date']
)
df_test['date'] = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

X1_test = dp.out_of_sample(steps=16)
X1_test.index.name = 'date'

y_pred = pd.DataFrame(
    model.predict(X1_test),
    index=X1_test.index,
    columns=y1.columns
)

y_test = df_test.unstack(['store_nbr', 'family'])
y_test = y_test.drop('id',axis=1)

X2_test = tsf(y_test)

y_pred_boosted = xgb.predict(X2_test) + y_pred.squeeze()

#STORE_NBR = 5  # 1 - 54
#FAMILY = 'PRODUCE'
# Uncomment to see a list of product families
# display(store_sales.index.get_level_values('family').unique())

#ax = y_pred_boosted['sales', STORE_NBR, FAMILY].plot()
#ax.set_title(f'{FAMILY} Sales at Store {STORE_NBR}');

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [3]:
y_submit = pd.DataFrame(y_pred_boosted).stack(['store_nbr','family'])

y_submit = y_submit.join(df_test.id).reindex(columns=['id','sales'])

y_submit.to_csv('submission1.csv', index=False)